In [11]:
import numpy as np
import cvxpy as cp

# Parameters for the optimization
size_param = 3
matrix_coeffs = np.array([[4, 0.9, 2], [1.3, 0.6, 6]])
limit_vals = np.array([1.2, 0.8])

# Variable declaration (positive)
var_opt = cp.Variable(size_param, pos=True)

# Objective: maximize entropy (equivalently minimize negative entropy)
entropy_obj = -cp.sum(cp.entr(var_opt))

# Constraint definitions
constr_set = [
    matrix_coeffs @ var_opt <= limit_vals,
    cp.sum(var_opt) == 1
]

# Define and solve optimization
entropy_problem = cp.Problem(cp.Minimize(entropy_obj), constr_set)

# Solve using default solver (SCS)
print("Initiating solution process...")
sol_result = entropy_problem.solve(solver=cp.SCS)

# Output solution results
print("Solution status:", entropy_problem.status)
print("Optimized negative entropy:", entropy_problem.value)
print("Optimized entropy:", -entropy_problem.value)
print("Optimal variable values:", var_opt.value)

# Constraint verification
print("\nChecking constraint adherence:")
constraint_eval = matrix_coeffs @ var_opt.value
print(f"{constraint_eval} <= {limit_vals}")
print(f"Constraints satisfied: {np.all(constraint_eval <= limit_vals + 1e-6)}")
print(f"Variable sum verification (should be 1): {np.sum(var_opt.value)} (Deviation: {abs(np.sum(var_opt.value) - 1)})")

# Dual variable retrieval
print("\nDual variables from optimization:")
dual_matrix = constr_set[0].dual_value
dual_sum_constraint = constr_set[1].dual_value
print("Dual (matrix constraints):", dual_matrix)
print("Dual (sum constraint):", dual_sum_constraint)

# Alternative method with scipy
print("\n\nAlternative method implementation:")
from scipy.optimize import minimize

# Negative entropy objective function
def alt_obj_function(y):
    y_safe = np.maximum(y, 1e-10)
    return np.sum(y_safe * np.log(y_safe))

# Constraint functions definition
def alt_sum_constraint(y):
    return np.sum(y) - 1

def alt_matrix_constraint(y):
    return limit_vals - matrix_coeffs @ y

# Initial guess
init_guess = np.ones(size_param) / size_param

# Constraints setup
alt_constraints = [
    {'type': 'eq', 'fun': alt_sum_constraint},
    {'type': 'ineq', 'fun': alt_matrix_constraint}
]

# Bounds ensuring positivity
var_bounds = [(1e-10, None)] * size_param

# Optimization using scipy
scipy_result = minimize(alt_obj_function, init_guess, method='SLSQP', bounds=var_bounds, constraints=alt_constraints)

print("Alternative solver status:", scipy_result.success)
print("Alternative optimal variables:", scipy_result.x)
print("Alternative method objective:", scipy_result.fun)
print("Constraints verification (scipy):")
print(f"Sum equals 1: {np.sum(scipy_result.x)}")
print(f"Matrix constraints: {matrix_coeffs @ scipy_result.x} <= {limit_vals}")


Initiating solution process...
Solution status: optimal
Optimized negative entropy: -0.4140802918747302
Optimized entropy: 0.4140802918747302
Optimal variable values: [0.08766211 0.8866622  0.02567569]

Checking constraint adherence:
[1.1999958  0.80001219] <= [1.2 0.8]
Constraints satisfied: False
Variable sum verification (should be 1): 0.9999999996762095 (Deviation: 3.2379054992759393e-10)

Dual variables from optimization:
Dual (matrix constraints): [0.62719043 0.52812277]
Dual (sum constraint): -1.761052441941864


Alternative method implementation:
Alternative solver status: True
Alternative optimal variables: [0.08766437 0.88666249 0.02567314]
Alternative method objective: -0.41407648951055376
Constraints verification (scipy):
Sum equals 1: 0.9999999999999999
Matrix constraints: [1.2 0.8] <= [1.2 0.8]


In [12]:
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize

# Setup parameters
dim = 3
coeff_matrix = np.array([[4, 0.9, 2], [1.3, 0.6, 6]])
limits = np.array([1.2, 0.8])

# Columns extraction
col_1 = coeff_matrix[:, 0]
col_2 = coeff_matrix[:, 1]
col_3 = coeff_matrix[:, 2]
col_array = [col_1, col_2, col_3]

# Dual objective definition
def dual_func(gamma):
    linear_part = -np.dot(limits, gamma)
    exp_sum = np.sum([np.exp(-np.dot(col_array[i], gamma)) for i in range(dim)])
    log_part = -np.log(exp_sum)
    return -(linear_part + log_part)

# Gradient definition for dual objective
def dual_grad(gamma):
    grad_linear = -limits
    exp_sum = 0
    weighted_sum = np.zeros_like(gamma)
    for i in range(dim):
        temp_exp = np.exp(-np.dot(col_array[i], gamma))
        exp_sum += temp_exp
        weighted_sum += col_array[i] * temp_exp
    grad_log = weighted_sum / exp_sum
    return -(grad_linear + grad_log)

# Constraint gamma >= 0
dual_bounds = [(0, None) for _ in range(len(limits))]

# Initial guess
gamma_start = np.ones(len(limits))

# Dual optimization
opt_dual = minimize(dual_func, gamma_start, method='L-BFGS-B', jac=dual_grad, bounds=dual_bounds)

# Dual results
print("Dual problem results:")
print("Solution status:", "Optimal" if opt_dual.success else "Not Optimal")
print("Optimal gamma:", opt_dual.x)
print("Optimal dual value:", -opt_dual.fun)

# Recover primal solution from dual
exp_denom = np.sum([np.exp(-np.dot(col_array[i], opt_dual.x)) for i in range(dim)])
primal_from_dual = np.array([np.exp(-np.dot(col_array[i], opt_dual.x)) / exp_denom for i in range(dim)])

print("\nDerived primal solution:")
print("Primal variables:", primal_from_dual)

# Primal objective computation
primal_obj_val = np.sum([x_i * np.log(x_i) for x_i in primal_from_dual if x_i > 0])
print("Primal objective value (from dual):", primal_obj_val)

# Constraint verification
print("\nConstraints verification:")
print("Matrix constraint evaluation:", coeff_matrix @ primal_from_dual, "<=", limits)
print("All constraints valid:", np.all(coeff_matrix @ primal_from_dual <= limits + 1e-6))
print("Sum of variables equals 1:", np.sum(primal_from_dual))

# Direct primal problem optimization
def primal_obj(x):
    x_adj = np.maximum(x, 1e-10)
    return np.sum(x_adj * np.log(x_adj))

def primal_sum_constraint(x):
    return np.sum(x) - 1

def primal_matrix_constraint(x):
    return limits - coeff_matrix @ x

# Primal constraints
primal_cons = [
    {'type': 'eq', 'fun': primal_sum_constraint},
    {'type': 'ineq', 'fun': primal_matrix_constraint}
]

# Primal bounds
primal_bounds = [(1e-10, None)] * dim

# Initial primal guess
primal_init = np.full(dim, 1/dim)

# Solve primal directly
primal_opt_res = minimize(primal_obj, primal_init, method='SLSQP', bounds=primal_bounds, constraints=primal_cons)

print("\nDirect primal optimization results:")
print("Solution status:", primal_opt_res.success)
print("Optimal primal variables:", primal_opt_res.x)
print("Optimal primal objective:", primal_opt_res.fun)

# Compare primal and dual solutions
print("\nComparison of primal and dual:")
print("Direct primal value:", primal_opt_res.fun)
print("Dual-derived primal value:", primal_obj_val)
print("Dual optimal value:", -opt_dual.fun)
print("Difference (Primal - Dual):", primal_opt_res.fun - (-opt_dual.fun))
print("Strong duality confirmed:", abs(primal_opt_res.fun - (-opt_dual.fun)) < 1e-5)


Dual problem results:
Solution status: Optimal
Optimal gamma: [0.62716766 0.52816966]
Optimal dual value: -0.41407648870949143

Derived primal solution:
Primal variables: [0.08766516 0.88666126 0.02567358]
Primal objective value (from dual): -0.4140798716482234

Constraints verification:
Matrix constraint evaluation: [1.20000292 0.80000294] <= [1.2 0.8]
All constraints valid: False
Sum of variables equals 1: 0.9999999999999999

Direct primal optimization results:
Solution status: True
Optimal primal variables: [0.08766437 0.88666249 0.02567314]
Optimal primal objective: -0.41407648951055376

Comparison of primal and dual:
Direct primal value: -0.41407648951055376
Dual-derived primal value: -0.4140798716482234
Dual optimal value: -0.41407648870949143
Difference (Primal - Dual): -8.010623275822581e-10
Strong duality confirmed: True


In [13]:
import numpy as np
from scipy.optimize import minimize

# Optimization parameters
dimension = 3
matrix_A = np.array([[4, 0.9, 2], [1.3, 0.6, 6]])
vector_b = np.array([1.2, 0.8])

# ---- Primal Optimization Problem ----
# Objective (negative entropy)
def primal_entropy(u):
    u_stable = np.maximum(u, 1e-10)
    return np.sum(u_stable * np.log(u_stable))

# Constraints
def sum_constraint(u):
    return np.sum(u) - 1

def linear_constraints(u):
    return vector_b - matrix_A @ u

# Initial guess
initial_u = np.full(dimension, 1/dimension)

# Constraint definitions
primal_constraints = [
    {'type': 'eq', 'fun': sum_constraint},
    {'type': 'ineq', 'fun': linear_constraints}
]

# Bounds to keep positivity
primal_bounds = [(1e-10, None)] * dimension

# Solve primal
primal_sol = minimize(primal_entropy, initial_u, method='SLSQP', bounds=primal_bounds, constraints=primal_constraints)
u_star = primal_sol.x
print("Primal solution u*:", u_star)
print("Primal objective value:", primal_sol.fun)

# ---- Dual Optimization Problem ----
# Columns of A for convenience
columns_A = [matrix_A[:, i] for i in range(dimension)]

# Dual objective
def dual_obj(alpha):
    linear_part = -np.dot(vector_b, alpha)
    exp_sum = sum(np.exp(-np.dot(columns_A[i], alpha)) for i in range(dimension))
    log_part = -np.log(exp_sum)
    return -(linear_part + log_part)

# Dual gradient
def dual_obj_grad(alpha):
    linear_grad = -vector_b
    exp_sum = sum_exp_weighted = 0
    for i in range(dimension):
        exp_term = np.exp(-np.dot(columns_A[i], alpha))
        exp_sum += exp_term
        sum_exp_weighted += columns_A[i] * exp_term
    log_grad = sum_exp_weighted / exp_sum
    return -(linear_grad + log_grad)

# Dual bounds (alpha >= 0)
dual_bounds = [(0, None) for _ in vector_b]

# Initial dual guess
initial_alpha = np.ones(len(vector_b))

# Solve dual
dual_sol = minimize(dual_obj, initial_alpha, method='L-BFGS-B', jac=dual_obj_grad, bounds=dual_bounds)
alpha_star = dual_sol.x
print("\nDual solution α*:", alpha_star)
print("Dual objective value:", -dual_sol.fun)

# ---- Dual Variable Calculation (nu) ----
nu_star = np.mean([-np.log(u_star[i]) - 1 - np.dot(columns_A[i], alpha_star) for i in range(dimension)])
print("\nDual variable ν*:", nu_star)

# ---- KKT Conditions Verification ----
print("\n--- KKT Conditions Verification ---")

# Stationarity condition
stationarity_check = (np.log(u_star) + 1) + matrix_A.T @ alpha_star + nu_star
print("1. Stationarity result:", stationarity_check)
print("   Stationarity satisfied:", np.allclose(stationarity_check, 0, atol=1e-5))

# Primal feasibility
ax_star = matrix_A @ u_star
print("\n2. Primal feasibility:")
print("   a. Matrix constraint Ax ≤ b:", ax_star, "≤", vector_b)
print("      Satisfied:", np.all(ax_star <= vector_b + 1e-6))

sum_u_star = np.sum(u_star)
print("   b. Sum constraint sum(u) = 1:", sum_u_star)
print("      Satisfied:", abs(sum_u_star - 1) < 1e-6)

print("   c. Positivity constraint u > 0:", u_star)
print("      Satisfied:", np.all(u_star > 0))

# Dual feasibility
print("\n3. Dual feasibility (α* ≥ 0):", alpha_star)
print("   Satisfied:", np.all(alpha_star >= 0))

# Complementary slackness
slackness = vector_b - ax_star
complementary = alpha_star * slackness
print("\n4. Complementary slackness α*(b - Ax):", complementary)
print("   Satisfied:", np.allclose(complementary, 0, atol=1e-5))

# ---- Strong Duality Check ----
print("\n--- Strong Duality Check ---")
primal_value = primal_sol.fun
dual_value = -dual_sol.fun
duality_gap = primal_value - dual_value
print("Primal optimal value:", primal_value)
print("Dual optimal value:", dual_value)
print("Duality gap:", duality_gap)
print("Strong duality confirmed:", abs(duality_gap) < 1e-5)


Primal solution u*: [0.08766437 0.88666249 0.02567314]
Primal objective value: -0.41407648951055376

Dual solution α*: [0.62716766 0.52816966]
Dual objective value: -0.41407648870949143

Dual variable ν*: -1.761052176946432

--- KKT Conditions Verification ---
1. Stationarity result: [-7.13752891e-07  9.64067460e-06 -8.92692171e-06]
   Stationarity satisfied: True

2. Primal feasibility:
   a. Matrix constraint Ax ≤ b: [1.2 0.8] ≤ [1.2 0.8]
      Satisfied: True
   b. Sum constraint sum(u) = 1: 0.9999999999999999
      Satisfied: True
   c. Positivity constraint u > 0: [0.08766437 0.88666249 0.02567314]
      Satisfied: True

3. Dual feasibility (α* ≥ 0): [0.62716766 0.52816966]
   Satisfied: True

4. Complementary slackness α*(b - Ax): [ 1.37168553e-09 -2.18091447e-09]
   Satisfied: True

--- Strong Duality Check ---
Primal optimal value: -0.41407648951055376
Dual optimal value: -0.41407648870949143
Duality gap: -8.010623275822581e-10
Strong duality confirmed: True
